# TASK_5

In [460]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [461]:
initial_data = pd.read_csv('initial_data.csv', index_col='id')

In [462]:
initial_cols = ['vendor_id', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'trip_duration']
initial_data = initial_data[initial_cols]
initial_data.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
id,,,,,,,
id2875421,2,1,-73.982155,40.767937,-73.964630,40.765602,455.0
id2377394,1,1,-73.980415,40.738564,-73.999481,40.731152,663.0
id3858529,2,1,-73.979027,40.763939,-74.005333,40.710087,2124.0
id3504673,2,1,-74.010040,40.719971,-74.012268,40.706718,429.0
id2181028,2,1,-73.973053,40.793209,-73.972923,40.782520,435.0


In [463]:
initial_data['log_trip_duration'] = np.log1p(initial_data['trip_duration'])
initial_data = initial_data.drop('trip_duration', axis = 1)
initial_data.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,log_trip_duration
id,,,,,,,
id2875421,2,1,-73.982155,40.767937,-73.964630,40.765602,6.122493
id2377394,1,1,-73.980415,40.738564,-73.999481,40.731152,6.498282
id3858529,2,1,-73.979027,40.763939,-74.005333,40.710087,7.661527
id3504673,2,1,-74.010040,40.719971,-74.012268,40.706718,6.063785
id2181028,2,1,-73.973053,40.793209,-73.972923,40.782520,6.077642


In [464]:
from sklearn.model_selection import train_test_split

In [465]:
X = initial_data.drop('log_trip_duration', axis=1)
Y = initial_data['log_trip_duration']
X.shape

(1458644, 6)

In [466]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X, Y, test_size=0.2, random_state=42)

In [467]:
X_train1

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
id,,,,,,
id0458976,2,1,-73.862762,40.768822,-73.891701,40.746689
id0434613,2,1,-73.958038,40.783237,-73.975510,40.760853
id3809234,2,1,-73.969460,40.785519,-73.989243,40.771748
id1203705,1,1,-73.981743,40.736549,-73.998352,40.726440
id1896645,2,1,-73.977913,40.752609,-73.975647,40.733139
...,...,...,...,...,...,...
id1753868,2,2,-73.955032,40.777328,-74.006203,40.749424
id2204747,1,1,-73.973618,40.763920,-73.983849,40.749874
id3043606,2,1,-74.006195,40.734283,-73.949608,40.785282


In [468]:
from sklearn.model_selection import KFold

splitter = KFold(n_splits=20, shuffle=True, random_state=33)

In [469]:
splitter.split(X_train1)

<generator object _BaseKFold.split at 0x000001A9A7EA54A0>

In [470]:
losses_test = []
losses_train = []

for train_index, test_index in splitter.split(X_train1):
    X_train, X_test = X_train1.values[train_index], X_train1.values[test_index]
    Y_train, Y_test = Y_train1.values[train_index], Y_train1.values[test_index]
    
    model = LinearRegression()
    model.fit(X_train, Y_train)
    
    losses_test.append(np.mean((model.predict(X_test)-Y_test)**2))
    losses_train.append(np.mean((model.predict(X_train)-Y_train)**2))

In [471]:
X_train.shape

(1108570, 6)

In [472]:
print(losses_test, losses_train)

[0.6662086054966643, 0.6038429437853066, 0.6154430241538906, 0.604062828336285, 0.6069934454514391, 0.590927916534856, 0.6202768968732896, 0.600061354790117, 0.6129546454790292, 0.6154911149897598, 0.6004858107948253, 0.605457758180123, 0.6057251219713804, 0.6104045238379128, 0.6238037332813571, 0.6132349425257343, 0.6041684116965784, 0.6623807542701925, 0.6030933265555022, 0.6034210516527898] [0.6070290128320978, 0.6089802331841703, 0.6083663955350783, 0.6089709202701666, 0.6088130650892979, 0.6096599814584411, 0.6081477057501963, 0.6091772827192719, 0.6085037728893131, 0.608391897786081, 0.6091581628398126, 0.6088947802565462, 0.6088744891130307, 0.608653963820331, 0.6080246483994879, 0.6084867302689955, 0.6089612104715889, 0.6071831017381298, 0.6090194294133254, 0.609004953620935]


In [473]:
dif_mean_er = np.mean(losses_test) - np.mean(losses_train)
dif_mean_er

0.0048068236600365966

In [474]:
print(dif_mean_er/np.mean(losses_train), dif_mean_er/np.mean(losses_test))

0.00789796993816734 0.007836080807516525


In [475]:
print(np.mean(losses_test + losses_train))

0.6110184987028333


In [476]:
print(np.mean(losses_test), np.mean(losses_train))

0.6134219105328516 0.608615086872815


In [477]:
mod = LinearRegression()
mod.fit(X_train1.values, Y_train1.values)
model_predictions = model.predict(X_test1.values)
np.mean((model_predictions-Y_test1.values)**2)

0.606022720537328

In [478]:
print(mod.coef_, mod.intercept_)

[ 0.02384988  0.00943766  1.32377541 -2.50183509 -0.12544093 -1.84988328] 272.39753573859537


# TASK_6

In [479]:
df = pd.read_csv('processed_data.csv', index_col = 'id')

In [480]:
df.head()

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,
id2875421,1,930.399753,0,455.0,1.500479
id2377394,0,930.399753,0,663.0,1.807119
id3858529,1,930.399753,0,2124.0,6.392080
id3504673,1,930.399753,0,429.0,1.487155
id2181028,1,930.399753,0,435.0,1.189925


In [481]:
df = df.assign(log_trip_duration = np.log1p(df['trip_duration']))
df = df.drop('trip_duration', axis = 1)

In [482]:
df.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration
id,,,,,
id2875421,1,930.399753,0,1.500479,6.122493
id2377394,0,930.399753,0,1.807119,6.498282
id3858529,1,930.399753,0,6.392080,7.661527
id3504673,1,930.399753,0,1.487155,6.063785
id2181028,1,930.399753,0,1.189925,6.077642


In [483]:
X_2 = df.drop('log_trip_duration', axis = 1)
Y_2 = df['log_trip_duration']



In [484]:
from sklearn.model_selection import train_test_split

In [485]:
#X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X_2, Y_2, test_size=0.2, random_state=42)

In [486]:
test_indexes = X_test1.index
train_indexes = X_train1.index

X_train_2 = X_2[X_2.index.isin(train_indexes)]
Y_train_2 = Y_2[Y_2.index.isin(train_indexes)]

X_test_2 = X_2[X_2.index.isin(test_indexes)]
Y_test_2 = Y_2[Y_2.index.isin(test_indexes)]

In [487]:
X_train_2.shape

(1166915, 4)

In [488]:
Y_test_2.shape

(291729,)

In [489]:
from sklearn.model_selection import KFold

In [493]:
kf = KFold(n_splits = 20, shuffle=False)
losses_test1 = []
losses_train1 = []

for train_indexes, test_indexes in kf.split(X_train_2):
    X_train_22, X_test_22 = X_2.values[train_indexes], X_2.values[test_indexes]
    Y_train_22, Y_test_22 = Y_2.values[train_indexes], Y_2.values[test_indexes]
    
    mod2 = LinearRegression()
    mod2.fit(X_train_22, Y_train_22)
    
    losses_test1.append(np.mean((mod2.predict(X_test_22) - Y_test_22)**2))
    losses_train1.append(np.mean((mod2.predict(X_train_22) - Y_train_22)**2))

In [494]:
print(np.mean(losses_test1))
print(np.mean(losses_train1))

0.43460725891662627
0.43186293658710745


In [492]:
mod = LinearRegression()
mod.fit(X_train_2, Y_train_2)
model_predictions = mod.predict(X_test_2)
np.mean((model_predictions - Y_test_2)**2)

0.4074669546810228